In [1]:
import os



In [2]:
import pandas as pd

# Path to the main directory
main_dir = 'data'
documents = []

# Traverse through the directory
for root, dirs, files in os.walk(main_dir):
    for file in files:
        if file.endswith('.md'):  # Check for text files
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:  # Use utf-8 encoding
                documents.append(f.read())


In [3]:
def chunk_text(text, chunk_size=500):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = [chunk_text(doc) for doc in documents]
chunks = [chunk for sublist in chunks for chunk in sublist]  # Flatten the list


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(chunks).toarray()


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'chunks' contains your text chunks
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(chunks).toarray()

import faiss

# Get the dimensionality of the vectors
dimension = vectors.shape[1]

# Initialize the FAISS index for L2 distance
index = faiss.IndexFlatL2(dimension)

# Add the vectors to the index
index.add(vectors.astype('float32'))



In [20]:
def retrieve_chunks(query, k=3):
    query_vector = vectorizer.transform([query]).toarray().astype('float32')
    distances, indices = index.search(query_vector, k)
    return [chunks[i] for i in indices[0]]

query = "What's sagemaker?"
relevant_chunks = retrieve_chunks(query)


In [7]:
context = " ".join(relevant_chunks)


In [21]:
from openai import OpenAI

api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(
    api_key=api_key
)


response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": f"{context}\n\n{query}"}
    ]
)

answer = response.choices[0].message.content
print(answer)


Amazon SageMaker is a fully managed service that provides developers and data scientists with the ability to build, train, and deploy machine learning (ML) models quickly. SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high quality models.
